In [1]:
import os 
import sys
from pathlib import Path
import argparse
from bs4 import BeautifulSoup
import json
import requests
import time
from datetime import datetime
import pandas as pd
import numpy as np 


In [59]:
url = "https://sportsbook.draftkings.com/leagues/basketball/88670846"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
# # # soup2 =  BeautifulSoup(page.content, "html.parser")

In [60]:
soup.find_all("div", class_="parlay-card-10-a")[0].find_all("span", text="Today")


[<span>Today</span>]

In [61]:
page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")

lines_by_day = soup.find_all("div", class_="parlay-card-10-a")
if len(lines_by_day[0].find_all("span", text="Today")) > 0: # we have lines left for today
    print("today lines")
    todays_lines = lines_by_day[0]
else: # no more lines for today (or potentially it's past midnight and the lines are listed as yesterday's) 
    hour = datetime.now().hour
    if hour < 3 and len(lines_by_day) > 1: # past midnight and we have two sections of lines
        # TODO: should replace <len(lines_by_day)> with some search for a tag that the first section is a date (representing the day that just finished)
        todays_lines = lines_by_day[0]
    else:
        print("no more lines")
        pass
    

today lines


In [33]:
lines_by_day[0].find_all("span", text="Today")


[]

In [76]:
## Parse the page
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

# ## Get today's lines
# lines_by_day = soup.find_all("div", class_="parlay-card-10-a")
# if len(lines_by_day[0].find_all("span", text="Today")) > 0: # we have lines left for today
#     todays_lines = lines_by_day[0]
# else: # no more lines for today (or potentially it's past midnight and the lines are listed as yesterday's) 
#     hour = datetime.now().hour
#     if hour < 3 and len(lines_by_day) > 1: # past midnight and we have two sections of lines
#         # TODO: should replace <len(lines_by_day)> with some search for a tag that the first section is a date (representing the day that just finished)
#         todays_lines = lines_by_day[0]
#     else:
#         print("no more lines")
#         break

## Get today's lines
x = soup.find_all("div", class_="parlay-card-10-a")[1]

## Get all Lines
lines = x.find_all("tbody", class_="sportsbook-table__body")[0].find_all("tr")

## Get the teams
teams = [line.find("div", class_="event-cell__name-text").text.split(" ")[0] for line in lines]

## Get the time
clocks = [line.find("div", class_='event-cell__clock') for line in lines[::2]]
times = [clock.find_all("span")[0].string if clock else None for clock in clocks]
quarters = [clock.find_all("span")[1].string[0] if clock else None for clock in clocks]

## Get the scores
scores = [line.find("span", class_="event-cell__score") for line in lines]
scores = [score.text if score else None for score in scores]

## Get the lines
mlines = [line.find("span", class_="sportsbook-odds american no-margin default-color") for line in lines]        
mlines = [line.text.replace("+", "") if line else None for line in mlines]
